In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = ""
path = '/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/'
os.chdir(path)
import time
FOLDERNAME = "TED-OpenNMT-sent2vec-0.9-" + str(time.strftime("%Y%m%d-%H%M"))
!mkdir $FOLDERNAME

path = path + FOLDERNAME
os.chdir(path)
!pwd

# import os
# path = '/content/drive/Shared drives/chinh-share/nmt-v3/OpenNMT-TED-EM-bert-ratio-8-2-2-20210128-0637'
# os.chdir(path)
# !pwd

/content/drive/Shared drives/chinh-share/nmt-v5-Faiss/TED-OpenNMT-sent2vec-0.9-20210410-0744


In [3]:
!nvidia-smi

Sat Apr 10 07:44:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## **Install libraries**

In [4]:
!pip install OpenNMT-py==1.2.0
!pip install -U scikit-learn

     |████████████████████████████████| 204kB 15.2MB/s 
     |████████████████████████████████| 2.6MB 13.0MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for configargparse: filename=ConfigArgParse-1.4-cp37-none-any.whl size=19638 sha256=f677d44a651965bd1ee1f73ff27341bcd60cf96543fc0024be2ac9190a173e6b
  Stored in directory: /root/.cache/pip/wheels/d6/61/f7/626bbd080a9f2f70015f92025e0af663c595146083f3d9aa05
Successfully built configargparse
  Found existing installation: torchtext 0.9.1
    Uninstalling torchtext-0.9.1:
      Successfully uninstalled torchtext-0.9.1
     |████████████████████████████████| 22.3MB 1.6MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!wget https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/en_vi_iwslt_sent2vec.tar.gz
!mkdir data_bin
!tar -xvf 'en_vi_iwslt_sent2vec.tar.gz'

--2021-04-10 07:44:27--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/dataset/en_vi_iwslt_sent2vec.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22512408 (21M) [application/octet-stream]
Saving to: ‘en_vi_iwslt_sent2vec.tar.gz’

en_vi_iwslt_sent2ve 100%[===================>]  21.47M  78.6MB/s    in 0.3s    

2021-04-10 07:44:28 (78.6 MB/s) - ‘en_vi_iwslt_sent2vec.tar.gz’ saved [22512408/22512408]

en_train_EM_score_0.95
vi_valid
en_train_EM_0.95
en_train_EM_factor_0.85
en_train_EM_score_0.8
vi_train
en_train_EM_factor_0.8
en_train_EM_0.8
en_valid
en_train_EM_factor_0.95
en_train
en_train_EM_score_0.85
vi_test
en_train_EM_0.85
en_train_EM_score_0.9
en_test
en_train_EM_factor_0.9
en_train_EM_0.9


In [6]:
!mkdir -p output
!onmt_preprocess -train_src 'en_train_EM_0.9' \\
-train_tgt 'vi_train' \\
-valid_src 'en_valid' \\
-valid_tgt 'vi_valid' \\
-save_data 'output/en-vi' 

[2021-04-10 07:44:33,376 INFO] Extracting features...
[2021-04-10 07:44:33,380 INFO]  * number of source features: 0.
[2021-04-10 07:44:33,380 INFO]  * number of target features: 0.
[2021-04-10 07:44:33,380 INFO] Building `Fields` object...
[2021-04-10 07:44:33,380 INFO] Building & saving training data...
[2021-04-10 07:44:33,550 INFO] Building shard 0.
[2021-04-10 07:44:36,958 INFO]  * saving 0th train data shard to output/en-vi.train.0.pt.
[2021-04-10 07:44:40,148 INFO]  * tgt vocab size: 18250.
[2021-04-10 07:44:40,198 INFO]  * src vocab size: 40209.
[2021-04-10 07:44:40,383 INFO] Building & saving validation data...
[2021-04-10 07:44:40,503 INFO] Building shard 0.
[2021-04-10 07:44:40,739 INFO]  * saving 0th valid data shard to output/en-vi.valid.0.pt.


In [7]:
!mkdir -p model
!onmt_train -data 'output/en-vi' \\
-save_model 'model/en-vi' \\
-layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8 \\
-encoder_type transformer -decoder_type transformer -position_encoding \\
-train_steps 30000  -max_generator_batches 2 -dropout 0.1 -batch_size 4096 \\
-batch_type tokens -normalization tokens  -accum_count 2 -optim adam -adam_beta2 0.998 \\
-decay_method noam -warmup_steps 8000 -learning_rate 2 -max_grad_norm 0 -param_init 0 \\
-param_init_glorot -label_smoothing 0.1 -valid_steps 1000 -save_checkpoint_steps 1000 \\
-report_every 1000 -world_size 1 -gpu_ranks 0

[2021-04-10 07:44:43,345 INFO]  * src vocab size = 40209
[2021-04-10 07:44:43,345 INFO]  * tgt vocab size = 18250
[2021-04-10 07:44:43,345 INFO] Building model...
[2021-04-10 07:44:50,564 INFO] NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(40209, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_feat

In [8]:
!ls -al model model/

model:
total 29997315
-rw------- 1 root root 1023908031 Apr 10 09:27 en-vi_step_10000.pt
-rw------- 1 root root 1023908031 Apr 10 07:55 en-vi_step_1000.pt
-rw------- 1 root root 1023908031 Apr 10 09:37 en-vi_step_11000.pt
-rw------- 1 root root 1023908031 Apr 10 09:47 en-vi_step_12000.pt
-rw------- 1 root root 1023908031 Apr 10 09:57 en-vi_step_13000.pt
-rw------- 1 root root 1023908031 Apr 10 10:07 en-vi_step_14000.pt
-rw------- 1 root root 1023908031 Apr 10 10:18 en-vi_step_15000.pt
-rw------- 1 root root 1023908031 Apr 10 10:28 en-vi_step_16000.pt
-rw------- 1 root root 1023908031 Apr 10 10:38 en-vi_step_17000.pt
-rw------- 1 root root 1023908031 Apr 10 10:48 en-vi_step_18000.pt
-rw------- 1 root root 1023908031 Apr 10 10:59 en-vi_step_19000.pt
-rw------- 1 root root 1023908031 Apr 10 11:09 en-vi_step_20000.pt
-rw------- 1 root root 1023908031 Apr 10 08:05 en-vi_step_2000.pt
-rw------- 1 root root 1023908031 Apr 10 11:19 en-vi_step_21000.pt
-rw------- 1 root root 1023908031 Apr 10 1

In [9]:
!onmt_translate -model model/en-vi_step_30000.pt -src en_test -tgt vi_test -output predict.txt

[2021-04-10 12:51:42,494 INFO] Translating shard 0.
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
/usr/local/lib/python3.7/dist-packages/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [50], which does not match the required output shape [10, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an ou

In [10]:
!tail vi_test

Cà vạt thì loè loẹt .
và lí do là bởi vì có 2 lí do , theo tôi nghĩ
Ông thích nói về thiên tài tâm linh của lứa tuổi .
Chúng tôi đều là người Triều Tiên , nhưng đã trở nên rất khác nhau do hậu quả của 67 năm bị chia cắt .
Đó là cách bạn xử lý một vấn đề khi bạn nhìn thấy chúng và đó không chỉ là việc than phiền về vấn đề đó .
Tham vọng của các bạn được thoã mãn , nó rất đẹp .
Không có thứ nào trong những điều trên thực sự hữu ích bởi vì bạn đang điều trị những triệu chứng chứ không phải nguyên nhân của các vấn đề cơ bản ở Phi Châu .
Nhưng hiện nay nhiều người sống đến 90 hay 100 tuổi , trừ khi họ bắt tay quá nhiều hay làm những điều đại loại thế .
Nhưng quý vị phải có những công cụ đúng .
Những điều này là một phần cuộc đời ông và là những gì ông còn nhớ


In [11]:
!git clone https://github.com/OpenNMT/OpenNMT-py.git

Cloning into 'OpenNMT-py'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17114 (delta 7), reused 8 (delta 4), pack-reused 17085
Receiving objects: 100% (17114/17114), 273.05 MiB | 20.02 MiB/s, done.
Resolving deltas: 100% (12323/12323), done.
Checking out files: 100% (228/228), done.


In [12]:
!ls -al

total 93793
drwx------  2 root root     4096 Apr 10 07:44 data_bin
-rw-------  1 root root   996149 Apr 10 07:32 en_test
-rw-------  1 root root  8024744 Apr 10 07:32 en_train
-rw-------  1 root root  8204550 Apr 10 07:32 en_train_EM_0.8
-rw-------  1 root root  8103017 Apr 10 07:32 en_train_EM_0.85
-rw-------  1 root root  8053185 Apr 10 07:32 en_train_EM_0.9
-rw-------  1 root root  8031617 Apr 10 07:32 en_train_EM_0.95
-rw-------  1 root root  3323998 Apr 10 07:32 en_train_EM_factor_0.8
-rw-------  1 root root  3281394 Apr 10 07:32 en_train_EM_factor_0.85
-rw-------  1 root root  3260286 Apr 10 07:32 en_train_EM_factor_0.9
-rw-------  1 root root  3250950 Apr 10 07:32 en_train_EM_factor_0.95
-rw-------  1 root root   860286 Apr 10 07:32 en_train_EM_score_0.8
-rw-------  1 root root   860286 Apr 10 07:32 en_train_EM_score_0.85
-rw-------  1 root root   860286 Apr 10 07:32 en_train_EM_score_0.9
-rw-------  1 root root   860286 Apr 10 07:32 en_train_EM_score_0.95
-rw-------  1 root roo

In [13]:
!perl OpenNMT-py/tools/multi-bleu.perl vi_test < predict.txt

BLEU = 23.74, 58.9/33.0/19.4/11.8 (BP=0.919, ratio=0.923, hyp_len=225300, ref_len=244219)
